## Reading the ChEMBL dataset

http://www.dalkescientific.com/writings/diary/archive/2020/09/16/faster_gzip_reading_in_python.html

https://github.com/hengwei-chan/standard_smiles/blob/main/python_scripts/stand_struct.py

https://iwatobipen.wordpress.com/2021/05/04/read-sdf-with-multi-thread-rdkit-memo-chemoinformatics/

In [31]:
import os 
import gzip
from typing import List, Set, Callable, Union, Tuple
import pandas as pd 
import tqdm.notebook as tqdm
import numpy as np

In [32]:
import rdkit
from rdkit import Chem #This gives us most of RDkits's functionality
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules
IPythonConsole.ipython_useSVG=True  #SVG's tend to look nicer than the png counterparts
print(rdkit.__version__)

# Mute all errors except critical
Chem.WrapLogs()
lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

2021.03.5


In [33]:
type(Chem.MolFromSmiles('C'))

rdkit.Chem.rdchem.Mol

In [41]:
#https://github.com/hengwei-chan/standard_smiles/blob/main/utils.py
    
def mol_to_smiles(mol: Chem.Mol, canonical: bool = True) -> str:
    """Generate Smiles from mol.
    Parameters:
    ===========
    mol: the input molecule
    canonical: whether to return the canonical Smiles or not
    Returns:
    ========
    The Smiles of the molecule (canonical by default). NAN for failed molecules."""

    if mol is None:
        return np.nan
    try:
        smi = Chem.MolToSmiles(mol, canonical=canonical)
        return smi
    except:
        return np.nan


def smiles_to_mol(smiles: str) -> Chem.Mol:
    """Generate a RDKit Molecule from a Smiles.
    Parameters:
    ===========
    smiles: the input string
    Returns:
    ========
    The RDKit Molecule. If the Smiles parsing failed, NAN is returned instead.
    """

    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return mol
        return np.nan
    except:
        return np.nan


def add_mol_col(df: pd.DataFrame, smiles_col="Smiles") -> pd.DataFrame:
    """Add a column containing the RDKit Molecule.
    Parameters:
    ===========
    df: the input DataFrame
    smiles_col: the name of the column containing the Smiles
    Returns:
    ========
    A DataFrame with a column containing the RDKit Molecule.
    """

    df["Mol"] = df[smiles_col].apply(smiles_to_mol)
    return df


def drop_cols(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """Remove the list of columns from the dataframe.
    Listed columns that are not available in the dataframe are simply ignored."""
    df = df.copy()
    cols_to_remove = set(cols).intersection(set(df.keys()))
    df = df.drop(cols_to_remove, axis=1)
    return df

def get_value(str_val):
    """convert a string into float or int, if possible."""
    if not str_val:
        return np.nan
    try:
        val = float(str_val)
        if "." not in str_val:
            val = int(val)
    except ValueError:
        val = str_val
    return val

# Make a general reading file for any format 

def read_sdf(fn, keep_mols=True, merge_prop: str = None, merge_list: Union[List, Set] = None) -> pd.DataFrame:
    """Create a DataFrame instance from an SD file.
    The input can be a single SD file or a list of files and they can be gzipped (fn ends with `.gz`).
    If a list of files is used, all files need to have the same fields.
    The molecules will be converted to Smiles and can optionally be stored as a `Mol` column.
    Records with no valid molecule will be dropped.
    Parameters:
    ===========
    merge_prop: A property in the SD file on which the file should be merge
        during reading.
    merge_list: A list or set of values on which to merge.
        Only the values of the list are kept.
    Returns:
    ========
    A Pandas DataFrame containing the structures as Smiles.
    """

    d = {"Smiles": []}
    if keep_mols:
        d["Mol"] = []
    ctr = {x: 0 for x in ["In", "Out", "Fail_NoMol"]}
    if merge_prop is not None:
        ctr["NotMerged"] = 0
    first_mol = True
    sd_props = set()
    if not isinstance(fn, list):
        fn = [fn]
    for f in fn:
        do_close = True
        if isinstance(f, str):
            if f.endswith(".gz"):
                file_obj = gzip.open(f, mode="rb")
            else:
                file_obj = open(f, "rb")
        else:
            file_obj = f
            do_close = False
        reader = Chem.ForwardSDMolSupplier(file_obj)
        
        for mol_index, mol in enumerate(reader):
            ctr["In"] += 1
            
            if ctr["In"] == 1000: 
                break
                
            if not mol:
                ctr["Fail_NoMol"] += 1
                continue
                
            if first_mol:
                first_mol = False
                # Is the SD file name property used?
                name = mol.GetProp("_Name")
                if len(name) > 0:
                    has_name = True
                    d["Name"] = []
                else:
                    has_name = False
                    
                for prop in mol.GetPropNames():
                    sd_props.add(prop)
                    d[prop] = []
                    
            if merge_prop is not None:
                # Only keep the record when the `merge_prop` value is in `merge_list`:
                if get_value(mol.GetProp(merge_prop)) not in merge_list:
                    ctr["NotMerged"] += 1
                    continue
                    
            mol_props = set()
            ctr["Out"] += 1
            
            for prop in mol.GetPropNames():
                if prop in sd_props:
                    mol_props.add(prop)
                    d[prop].append(get_value(mol.GetProp(prop)))
                mol.ClearProp(prop)
            if has_name:
                d["Name"].append(get_value(mol.GetProp("_Name")))
                mol.ClearProp("_Name")

            # append NAN to the missing props that were not in the mol:
            missing_props = sd_props - mol_props
            for prop in missing_props:
                d[prop].append(np.nan)
            d["Smiles"].append(mol_to_smiles(mol))
            if keep_mols:
                d["Mol"].append(mol)
                
        if do_close:
            file_obj.close()
            
    # Make sure, that all columns have the same length.
    # Although, Pandas would also complain, if this was not the case.
    d_keys = list(d.keys())
    
    if len(d_keys) > 1:
        k_len = len(d[d_keys[0]])
        for k in d_keys[1:]:
            assert k_len == len(d[k]), f"{k_len=} != {len(d[k])}"
            
    result = pd.DataFrame(d)
    
    print(ctr)
    return result

In [38]:
x = read_sdf('curated_set_with_publication_year.sd.gz')

{'In': 1000, 'Out': 999, 'Fail_NoMol': 0}


In [40]:
x.columns

Index(['Smiles', 'Mol', 'TC_key', 'BIOACT_PCHEMBL_VALUE', 'TGT_CHEMBL_ID',
       'TGT_ORGANISM', 'TGT_TID', 'CMP_ACD_LOGD', 'CMP_ACD_LOGP', 'CMP_ALOGP',
       'CMP_AROMATIC_RINGS', 'CMP_CHEMBL_ID', 'CMP_FULL_MWT', 'CMP_HBA',
       'CMP_HBD', 'CMP_HEAVY_ATOMS', 'CMP_INORGANIC_FLAG', 'CMP_LOGP',
       'CMP_MED_CHEM_FRIENDLY', 'CMP_MOLREGNO', 'CMP_NUM_ALERTS',
       'CMP_NUM_RO5_VIOLATIONS', 'CMP_PSA', 'CMP_RO3_PASS', 'CMP_RTB',
       'CMP_STANDARD_INCHI_KEY', 'CMP_STRUCTURE_TYPE', 'DOC_YEAR',
       'CMP_MOLECULAR_SPECIES_ACID', 'CMP_MOLECULAR_SPECIES_BASE',
       'CMP_MOLECULAR_SPECIES_NEUTRAL', 'CMP_MOLECULAR_SPECIES_ZWITTERION',
       'CMP_TYPE_SMALL_MOLECULE', 'CMP_TYPE_PROTEIN'],
      dtype='object')

In [44]:
x.CMP_PSA

0      231.95
1      191.11
2       85.89
3       91.10
4       26.30
        ...  
994     89.94
995     99.17
996     89.94
997     76.14
998     76.14
Name: CMP_PSA, Length: 999, dtype: float64

In [ ]:
import gzip

def molgen(hnd):
    mol_text_tmp = ""
    _iter = 0
    while _iter < 10_000:
        line = hnd.readline()
        if not line:
            return
        line = line.decode("utf-8")
        mol_text_tmp += line
        if line.startswith("$$$$"):
            mol_text = mol_text_tmp
            mol_text_tmp = ""
            yield mol_text
        
        _iter = _iter + 1
        
with gzip.open("curated_set_with_publication_year.sd.gz", "rb") as gzip_hnd:
    for mol_text in molgen(gzip_hnd):
        #print(mol_text)
        suppl = Chem.SDMolSupplier()
        suppl.SetData(mol_text)
        mol = next(suppl)
        #print(mol.GetNumAtoms())
        #print("------------------")
